# RNN的Gluon实现

In [2]:
import sys
sys.path.append('../')

In [6]:
import gluonbook as gb
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time
import mxnet as mx

In [4]:
(corpus_indices,char_to_idx,idx_to_char,
vocab_size) = gb.load_data_jay_lyrics()

## 定义模型

* 下面构造一个单隐藏层、隐藏单元个数为256的rnn，并对权重进行初始化

In [7]:
num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize(ctx= mx.gpu())

* 接下来调用begin_state来返回初始化的隐藏状态列表。它有一个形状为 (隐藏层个数，批量大小，隐藏单元个数)的元素

In [10]:
batch_size = 2
state = rnn_layer.begin_state(batch_size = batch_size,ctx= mx.gpu())
state[0].shape

(1, 2, 256)

In [11]:
num_steps = 35
X = nd.random.uniform(shape=(num_steps,batch_size,vocab_size),ctx = mx.gpu())
Y ,state_new = rnn_layer(X,state)
Y.shape,len(state_new),state_new[0].shape

((35, 2, 256), 1, (1, 2, 256))

## 下面继承Block类来定义一个完整的循环神经网络。它首先将输入数据使用one-hot向量表示后输入到rnn-layer中，然后使用全连接层得到输出

In [12]:
class RNNModel(nn.Block):
    def __init__(self,rnn_layer,vocab_size,**kwargs):
        super(RNNModel,self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.dense = nn.Dense(vocab_size)
    
    def forward(self,input,state):
        #将输入转置成(num_steps,batch_size)后获取one-hot表示
        X = nd.one_hot(input.T,self.vocab_size)
        Y,state = self.rnn(X,state)
        # 全连接层会⾸先将 Y 的形状变成（num_steps * batch_size， num_hiddens），
        # 它的输出形状为（num_steps * batch_size， vocab_size）。
        output = self.dense(Y.reshape((-1,Y.shape[-1])))
        return output,state
    
    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

## 模型训练

 * 定义一个预测函数。这里的实现区别在于前向计算和初始化隐藏状态的函数接口

In [30]:
def predict_rnn_gluon(prefix,num_chars,model,vocab_size,ctx,idx_to_char,char_to_idx):
    #初始化隐藏层状态
    state = model.begin_state(batch_size  =1,ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    
    for t in range(num_chars+len(prefix)-1):
        #上一次的输出作为这一次的输入
        X = nd.array([output[-1]],ctx).reshape((1,1))
        (Y,state) = model(X,state) #前向运算,里面进行了one-hot编码
        if t <len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])
        

In [33]:
ctx = gb.try_gpu()
model = RNNModel(rnn_layer,vocab_size)
model.initialize(force_reinit=True,ctx=ctx)
predict_rnn_gluon('分开',10,model,vocab_size,ctx,idx_to_char,char_to_idx)

'分开深谢腿耍连学进漠泛飘'

## 接下来实现训练函数，这里使用了随机采样

In [50]:
def train_and_predict_rnn_gluon(model,num_hiddens,vocab_size,ctx,
                               corpus_indices,idx_to_char,char_to_idx,
                               num_epochs,num_steps,lr,clipping_theta,
                               batch_size,pred_period,pred_len,prefixes):
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx,init=init.Normal(0.01),force_reinit=True)
    trainer = gluon.Trainer(model.collect_params(),'sgd',{'learning_rate':lr,'momentum':0,'wd':0})
    
    for epoch in range(num_epochs):
        loss_sum,start = 0.0,time.time()
        data_iter = gb.data_iter_random(corpus_indices,batch_size,num_steps,ctx=ctx)
        #state = model.begin_state(batch_size=batch_size,ctx=ctx)
        for t,(X,Y) in enumerate(data_iter):
            state = model.begin_state(batch_size=batch_size,ctx=ctx)
#             for s in state:
#                 s.detach()
            with autograd.record():
                (output,state) = model(X,state)
                y = Y.T.reshape((-1,))
                l = loss(output,y).mean()
            l.backward()
            #梯度裁剪
            params = [p.data() for p in model.collect_params().values()]
            gb.grad_clipping(params,clipping_theta,ctx)
            trainer.step(1)
            loss_sum+=l.asscalar()
        if (epoch+1) % pred_period == 0:
            print('epoch %d, perplexity %f,time %.2f sec'%
                 (epoch+1,math.exp(loss_sum/(t+1)),time.time()-start))
            for prefix in prefixes:
                print('-',predict_rnn_gluon(prefix,pred_len,model,vocab_size,ctx,idx_to_char,char_to_idx))

In [51]:
num_epochs, batch_size, lr, clipping_theta = 1000, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 100, 50, ['分开', '不分开']
train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 100, perplexity 18.647071,time 0.07 sec
- 分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
- 不分开  我有你这里很 后知后觉 我该好好生活 后知后觉 你已经离开我 不知不觉 我跟好这生活 后知后觉 
epoch 200, perplexity 3.092964,time 0.07 sec
- 分开 有什么 干什么 我吸就这样牵着你的手不放开 爱可不能够永简单纯没有悲哀 我 想带你骑单车 默 还不
- 不分开扫 我不要再想 我不 我想 我不要再想你 不知不觉 你已经离开我 已知不觉 不知了觉截棍 哼哼哈兮 
epoch 300, perplexity 1.848787,time 0.07 sec
- 分开的凯萨琳公主泊专 离不开暴风圈来不及逃 我不能再想 我不能再想 我不要再想 我不要再想 我不要再想 
- 不分开扫 我叫能爸想 我不要再想 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 
epoch 400, perplexity 1.554516,time 0.08 sec
- 分开 默默不碎 泣不再 一壶令它心仪的母斑鸠 牛仔红蕃 在小镇 背对背决斗 一只灰狼 问候伦 废墟背囱 
- 不分开扫 然后将过去 慢慢温习 让我该恨都难以美后 将真心抽离写成日记 像是一场默剧 美无法被安排的雨 有
epoch 500, perplexity 1.415595,time 0.08 sec
- 分开 娘子已人三千七百多年 你在橱窗前 说视碑文的字眼 我却在旁静静欣赏 就想躲 说你眼睛看着我 不知不
- 不分开吗 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后人 
epoch 600, perplexity 1.417990,time 0.07 sec
- 分开 宁愿已经三千七百多年 你在橱窗前 凝视碑文的字眼 我却在旁静静欣赏后还会够 它色后的 快时光中的没
- 不分开期 我叫你爸 你打我妈 这样对吗干嘛这样 从必让酒牵鼻成走记 像是在场只敌 你的完美主义 太彻底 分
epoch 700, perplexity 1.320383,time 0.08 sec
- 分开 周今心 一步两武术的老板 

## Gluon实现速度快了很多很多